# 3. 평가 메트릭스 구현

이 챕터에서는 추천 시스템의 성능을 평가하기 위한 다양한 메트릭스를 구현합니다. RMSE(Root Mean Squared Error)를 통한 평점 예측 정확도와 Precision@K, Recall@K를 통한 추천 품질을 측정합니다.

## 3.1 필요한 라이브러리 불러오기

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from typing import Dict, List
from sklearn.metrics import mean_squared_error

# 상위 디렉토리 경로를 시스템 경로에 추가하여 utils 모듈을 import할 수 있게 함
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# utils 모듈에서 필요한 클래스 import
from utils.models import Metrics

## 3.2 평가 메트릭스 클래스 구현

In [ ]:
class MetricCalculator:
    def calc(self, true_rating: List[float], pred_rating: List[float],
             true_user2items: Dict[int, List[int]],
             pred_user2items: Dict[int, List[int]], k: int, params: Dict=None) -> Metrics:
        rmse = self._calc_rmse(true_rating, pred_rating)
        p = self._calc_precision_at_k(true_user2items, pred_user2items, k)
        r = self._calc_recall_at_k(true_user2items, pred_user2items, k)
        return Metrics(rmse, p, r, params or {})

    def _calc_rmse(self, y, yhat) -> float:
        """
        실제 평점과 예측 평점 사이의 Root Mean Squared Error를 계산합니다.
        
        Args:
            y: 실제 평점 리스트
            yhat: 예측 평점 리스트
            
        Returns:
            RMSE 값 (낮을수록 좋음)
        """
        if not y or not yhat: return np.nan
        return float(np.sqrt(mean_squared_error(y, yhat)))

    def _precision_at_k(self, true_items: List[int], pred_items: List[int], k: int) -> float:
        """
        단일 사용자에 대한 Precision@K를 계산합니다.
        추천 목록의 상위 K개 아이템 중 실제로 관련 있는 항목의 비율입니다.
        
        Args:
            true_items: 실제 관련 있는 아이템 ID 리스트
            pred_items: 예측/추천된 아이템 ID 리스트
            k: 상위 K개 항목만 고려
            
        Returns:
            Precision@K 값 (0~1 사이, 높을수록 좋음)
        """
        if k == 0: return 0.0
        return len(set(true_items) & set(pred_items[:k])) / k

    def _recall_at_k(self, true_items: List[int], pred_items: List[int], k: int) -> float:
        """
        단일 사용자에 대한 Recall@K를 계산합니다.
        실제 관련 있는 항목 중 추천 목록의 상위 K개 항목에 포함된 비율입니다.
        
        Args:
            true_items: 실제 관련 있는 아이템 ID 리스트
            pred_items: 예측/추천된 아이템 ID 리스트
            k: 상위 K개 항목만 고려
            
        Returns:
            Recall@K 값 (0~1 사이, 높을수록 좋음)
        """
        if len(true_items) == 0 or k == 0: return 0.0
        return len(set(true_items) & set(pred_items[:k])) / len(true_items)

    def _calc_precision_at_k(self, true_u2i, pred_u2i, k):
        """
        모든 사용자에 대한 평균 Precision@K를 계산합니다.
        """
        scores = []
        for u in true_u2i.keys():
            scores.append(self._precision_at_k(true_u2i[u], pred_u2i.get(u, []), k))
        return float(np.mean(scores)) if scores else 0.0

    def _calc_recall_at_k(self, true_u2i, pred_u2i, k):
        """
        모든 사용자에 대한 평균 Recall@K를 계산합니다.
        """
        scores = []
        for u in true_u2i.keys():
            scores.append(self._recall_at_k(true_u2i[u], pred_u2i.get(u, []), k))
        return float(np.mean(scores)) if scores else 0.0

## 3.3 메트릭스 사용 예제

In [ ]:
# 간단한 예제로 메트릭스 계산기 시연
# 실제 평점
true_ratings = [5.0, 4.0, 3.0, 2.0, 1.0]
# 예측 평점
pred_ratings = [4.5, 3.5, 3.5, 1.5, 2.0]

# 실제 사용자가 높이 평가한(관련 있는) 아이템 리스트
true_u2i = {1: [101, 102, 103], 2: [201, 202]}
# 추천된 아이템 목록
pred_u2i = {1: [101, 104, 103, 105], 2: [203, 201, 204]}

# 메트릭스 계산
calculator = MetricCalculator()
metrics = calculator.calc(true_ratings, pred_ratings, true_u2i, pred_u2i, k=3)

print(f"RMSE: {metrics.rmse:.3f}")
print(f"Precision@3: {metrics.precision_at_k:.3f}")
print(f"Recall@3: {metrics.recall_at_k:.3f}")

## 3.4 메트릭스 해석 방법

추천 시스템의 성능은 다음 메트릭스로 평가합니다:

1. **RMSE (Root Mean Squared Error)**:
   - 예측 평점의 정확도를 측정
   - 낮을수록 좋으며, 실제 평점과 예측 평점의 차이가 작음을 의미
   - 주로 평점 예측 정확도 비교에 사용

2. **Precision@K**:
   - 추천된 상위 K개 아이템 중 실제로 사용자가 관심 있는 비율
   - 높을수록 좋으며, 1.0이면 모든 추천이 관련성 있음
   - 추천의 정확성을 측정

3. **Recall@K**:
   - 사용자가 관심 있는 모든 아이템 중 상위 K개 추천에 포함된 비율
   - 높을수록 좋으며, 1.0이면 모든 관련 아이템이 추천됨
   - 추천의 완전성을 측정

이러한 메트릭스를 함께 고려하여 추천 시스템의 전체적인 성능을 평가합니다.